In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data
## Drop irrelevant features

data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Encode categorical variables
data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [7]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [10]:
## One-Hot Encode Geography
from sklearn.preprocessing import OneHotEncoder
ohe_geography = OneHotEncoder()
geo_encoder = ohe_geography.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [11]:
ohe_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geo_encoded_df = pd.DataFrame(geo_encoder.astype(int).toarray(), columns=ohe_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
9995,1,0,0
9996,1,0,0
9997,1,0,0
9998,0,1,0


In [14]:
data = pd.concat([data.drop(columns=['Geography']), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [ ]:
## Save the encoders
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ohe_geography.pkl', 'wb') as file:
    pickle.dump(ohe_geography, file)

In [16]:
## Divide data into independent and dependent features
X = data.drop(columns=['Exited'])
y = data['Exited']

In [17]:
## Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
## Scale these independent features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
## Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [28]:
X_train_scaled = pd.DataFrame(data=X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=X_test_scaled, columns=X_test.columns)

In [29]:
X_train_scaled

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0.356500,0.913248,-0.655786,0.345680,-1.218471,0.808436,0.649203,0.974817,1.367670,1.001501,-0.579467,-0.576388
1,-0.203898,0.913248,0.294938,-0.348369,0.696838,0.808436,0.649203,0.974817,1.661254,-0.998501,1.725723,-0.576388
2,-0.961472,0.913248,-1.416365,-0.695393,0.618629,-0.916688,0.649203,-1.025834,-0.252807,-0.998501,-0.579467,1.734942
3,-0.940717,-1.094993,-1.131148,1.386753,0.953212,-0.916688,0.649203,-1.025834,0.915393,1.001501,-0.579467,-0.576388
4,-1.397337,0.913248,1.625953,1.386753,1.057449,-0.916688,-1.540351,-1.025834,-1.059600,1.001501,-0.579467,-0.576388
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1.207474,0.913248,1.435808,1.039728,-0.102301,-0.916688,0.649203,0.974817,-0.539860,1.001501,-0.579467,-0.576388
7996,0.314989,-1.094993,1.816097,-1.389442,-1.218471,-0.916688,0.649203,0.974817,-1.733882,1.001501,-0.579467,-0.576388
7997,0.865009,-1.094993,-0.085351,-1.389442,-1.218471,2.533560,-1.540351,-1.025834,-0.142765,1.001501,-0.579467,-0.576388
7998,0.159323,0.913248,0.390011,1.039728,1.827259,-0.916688,0.649203,-1.025834,-0.050826,1.001501,-0.579467,-0.576388


In [30]:
X_test_scaled

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,-0.577496,0.913248,-0.655786,-0.695393,0.329937,0.808436,-1.540351,-1.025834,-1.019605,-0.998501,1.725723,-0.576388
1,-0.297297,0.913248,0.390011,-1.389442,-1.218471,0.808436,0.649203,0.974817,0.798883,1.001501,-0.579467,-0.576388
2,-0.525607,-1.094993,0.485083,-0.348369,-1.218471,0.808436,0.649203,-1.025834,-0.727980,-0.998501,-0.579467,1.734942
3,-1.511492,0.913248,1.911170,1.039728,0.689272,0.808436,0.649203,0.974817,1.221387,-0.998501,1.725723,-0.576388
4,-0.951094,-1.094993,-1.131148,0.692704,0.782839,-0.916688,0.649203,0.974817,0.247560,-0.998501,-0.579467,1.734942
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.515230,-1.094993,1.340735,-0.001345,0.354911,-0.916688,-1.540351,0.974817,-0.963100,-0.998501,1.725723,-0.576388
1996,-0.442586,0.913248,-1.321293,1.733777,-1.218471,-0.916688,-1.540351,0.974817,0.164545,1.001501,-0.579467,-0.576388
1997,0.813120,-1.094993,0.770301,0.692704,-1.218471,-0.916688,0.649203,-1.025834,-1.165916,1.001501,-0.579467,-0.576388
1998,0.418766,0.913248,-0.941003,-0.348369,-1.218471,-0.916688,0.649203,-1.025834,-0.411635,1.001501,-0.579467,-0.576388


In [33]:
y_train = pd.DataFrame(data=y_train, columns=['Exited'])
y_train

,Exited
9254,0
1561,0
1670,1
6087,1
6669,1
...,...
5734,0
5191,0
5390,1
860,1


In [34]:
y_test = pd.DataFrame(data=y_test, columns=['Exited'])
y_test

,Exited
6252,0
4684,0
1731,0
4742,0
4521,0
...,...
6412,1
8285,0
7853,1
1095,1


## ANN Implementation

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [37]:
(X_train_scaled.shape[1],)

(12,)

In [38]:
## Build our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  ## HL1 connected with input layer
    Dense(32, activation='relu'),  ## HL2
    Dense(1, activation='sigmoid')  ## O/p layer
])

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.01)

In [41]:
## Compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [74]:
## Set up the Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [75]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [76]:
## Train the model
history = model.fit(
    X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
    epochs=100, callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 0.2976 - accuracy: 0.8776 - val_loss: 0.3516 - val_accuracy: 0.8660
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.2934 - accuracy: 0.8774 - val_loss: 0.3913 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.2938 - accuracy: 0.8755 - val_loss: 0.3570 - val_accuracy: 0.8635
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.2903 - accuracy: 0.8784 - val_loss: 0.3714 - val_accuracy: 0.8515
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.2900 - accuracy: 0.8779 - val_loss: 0.3678 - val_accuracy: 0.8500
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.2895 - accuracy: 0.8792 - val_loss: 0.3706 - val_accuracy: 0.8530
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 0.2855 - accuracy: 0.8801 - val_loss: 0.3723 - val_accuracy: 0.8550

In [77]:
model.save('model.h5')

d:\DataAnalystBootcamp\End to End Projects\Deep Learning Project Using ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [78]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [79]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 3824), started 1:08:57 ago. (Use '!kill 3824' to kill it.)